`파일` > `드라이브에 사본 저장` 후 실습하세요.

# Word2Vec

In [ ]:
import gensim
gensim.__version__

'4.3.1'

In [ ]:
!pip install nltk==3.8.1

## 영어 데이터 다운로드 및 전처리

In [ ]:
import re
from lxml import etree
import urllib.request
import zipfile
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

Word2Vec을 학습하기 위해서 데이터를 다운로드합니다.

In [ ]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/GaoleMeng/RNN-and-FFNN-textClassification/master/ted_en-20160408.xml", filename="ted_en-20160408.xml")

('ted_en-20160408.xml', <http.client.HTTPMessage at 0x787a1de975e0>)

In [ ]:
targetXML = open('ted_en-20160408.xml', 'r', encoding='UTF8')
target_text = etree.parse(targetXML)

# xml 파일로부터 <content>와 </content> 사이의 내용만 가져온다.
parse_text = '\n'.join(target_text.xpath('//content/text()'))

# 정규 표현식의 sub 모듈을 통해 content 중간에 등장하는 (Audio), (Laughter) 등의 배경음 부분을 제거.
# 해당 코드는 괄호로 구성된 내용을 제거.
content_text = re.sub(r'\([^)]*\)', '', parse_text)

현재 영어 텍스트가 content_text에 저장되어져 있습니다. 이에 대해서 NLTK의 sent_tokenize를 통해서 문장을 구분해봅시다.

In [ ]:
len(content_text)

24062319

In [ ]:
# 입력 코퍼스에 대해서 NLTK를 이용하여 문장 토큰화를 수행.
sent_text = sent_tokenize(content_text)

In [ ]:
# 각 문장에 대해서 구두점을 제거하고, 대문자를 소문자로 변환.
normalized_text = []
for string in sent_text:
     tokens = re.sub(r"[^a-z0-9]+", " ", string.lower())
     normalized_text.append(tokens)

# 각 문장에 대해서 NLTK를 이용하여 단어 토큰화를 수행.
result = [word_tokenize(sentence) for sentence in normalized_text]

총 문장의 개수는 273,424개입니다.

In [ ]:
print('총 샘플의 개수 : {}'.format(len(result)))

총 샘플의 개수 : 273424


In [ ]:
for line in result[:3]: # 샘플 3개만 출력
    print(line)

['here', 'are', 'two', 'reasons', 'companies', 'fail', 'they', 'only', 'do', 'more', 'of', 'the', 'same', 'or', 'they', 'only', 'do', 'what', 's', 'new']
['to', 'me', 'the', 'real', 'real', 'solution', 'to', 'quality', 'growth', 'is', 'figuring', 'out', 'the', 'balance', 'between', 'two', 'activities', 'exploration', 'and', 'exploitation']
['both', 'are', 'necessary', 'but', 'it', 'can', 'be', 'too', 'much', 'of', 'a', 'good', 'thing']


result => 이중 리스트

[[a], [b], [c]]

## 영어 Word2Vec 훈련시키기

In [ ]:
from gensim.models import Word2Vec
model = Word2Vec(sentences=result, vector_size=100, window=5, min_count=5, workers=4, sg=0)

여기서 Word2Vec의 하이퍼파라미터값은 다음과 같습니다.  

vector_size = 워드 벡터의 특징 값. 즉, 임베딩 된 벡터의 차원.  
window = 컨텍스트 윈도우 크기  
min_count = 단어 최소 빈도 수 제한 (빈도가 적은 단어들은 학습하지 않는다.)  
workers = 학습을 위한 프로세스 수  
sg = 0은 CBOW, 1은 Skip-gram.  

코사인 유사도라는 것을 유사도 메트릭으로 사용. 값의 범위가 -1 ~ 1

In [ ]:
model_result = model.wv.most_similar("man")
print(model_result)

[('woman', 0.8596423864364624), ('guy', 0.812863826751709), ('lady', 0.7970998287200928), ('boy', 0.7578081488609314), ('girl', 0.7450876832008362), ('gentleman', 0.719805896282196), ('soldier', 0.7028768062591553), ('kid', 0.6901385188102722), ('poet', 0.664341151714325), ('son', 0.6367191076278687)]


In [ ]:
model.wv["man"]

array([ 0.6503617 , -2.4734154 ,  0.4829468 , -0.15286069,  0.9017836 ,
       -0.66970676,  0.8678353 ,  1.1511679 ,  0.0997102 ,  0.456802  ,
       -0.75599   ,  0.42009178, -0.6391357 ,  1.2612363 , -1.0570009 ,
       -0.50605303,  0.723856  ,  0.26778778,  1.2178475 , -0.34232962,
        1.1199574 ,  1.2822349 ,  0.27641374, -0.5073515 ,  0.6950679 ,
       -0.26846227, -2.728288  , -0.4461816 , -0.3391241 , -0.76207995,
        0.5201528 ,  0.56509805,  0.97161764,  1.0422541 , -0.9933656 ,
       -1.1154146 , -0.980676  ,  0.05117747, -1.4796218 , -0.3365385 ,
       -0.19874553, -1.3220725 , -0.56493443,  1.2749588 , -0.04715608,
       -1.4343884 , -1.4949763 , -1.7082741 , -0.68917245,  0.34738255,
       -0.913338  , -1.7175022 , -0.7159399 ,  1.42226   ,  1.1637232 ,
       -0.23693325, -0.986852  , -0.6388867 , -1.3195604 , -0.6922128 ,
       -0.57989174,  0.2144543 ,  1.2653288 ,  1.1477228 , -1.5327743 ,
        0.9178386 , -1.1651814 ,  0.62118435, -1.4506468 ,  2.38

In [ ]:
len(model.wv["man"])

100

In [ ]:
from gensim.models import KeyedVectors
model.wv.save_word2vec_format('eng_w2v') # 모델 저장
loaded_model = KeyedVectors.load_word2vec_format("eng_w2v") # 모델 로드

In [ ]:
model_result = loaded_model.most_similar("man")
print(model_result)

[('woman', 0.8596423864364624), ('guy', 0.812863826751709), ('lady', 0.7970998287200928), ('boy', 0.7578081488609314), ('girl', 0.7450876832008362), ('gentleman', 0.719805896282196), ('soldier', 0.7028768062591553), ('kid', 0.6901385188102722), ('poet', 0.664341151714325), ('son', 0.6367191076278687)]


In [ ]:
model.wv.vectors.shape

(21613, 100)

In [ ]:
# 현재 경로
%pwd

'/content'

## 한국어 데이터 다운로드 및 전처리

KoNLPy의 OKT 등은 형태소 분석 속도가 너무 느립니다. 그래서 Mecab을 설치하겠습니다.  
단, Mecab은 형태소 분석 속도는 빠르지만 설치하는데 시간이 좀 걸립니다.

In [ ]:
!pip install konlpy
!pip install mecab-python
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 40.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 39.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 581.6/581.6 kB 7.3 MB/s eta 0:00:00
  Created wheel for mecab-python: filename=mecab_python-1.0.0-py3-none-any.whl size=1231 sha256=4e6f1d22668a493eb529cf17adf6b22e58e405083e958c2cd6d72ade15f0fb63
  Stored in directory: /root/.cache/pip/wheels/ec/41/c8/ce8fb469be2547ce596c4000613806505dc54c296aae45da71
Successfully built mecab-python
Install mecab-ko
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 1381k  100 1381k    0     0   483k      0  0:00:02  0:00:02 --:--:-- 1250k
mecab-0.996-ko-0.9.2/
mecab-0.996-ko-0.9.2/example/
mecab-0.996-ko-0.9.2/example/ex

In [ ]:
import urllib.request
from konlpy.tag import Mecab
from gensim.models.word2vec import Word2Vec
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings.txt", filename="ratings.txt")

('ratings.txt', <http.client.HTTPMessage at 0x7879e50d9810>)

In [ ]:
train_data = pd.read_table('ratings.txt')

In [ ]:
train_data[:5] # 상위 5개 출력

,id,document,label
0,8112052,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1
1,8132799,"디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산...",1
2,4655635,폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.,1
3,9251303,와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런...,1
4,10067386,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화.,1


In [ ]:
print(len(train_data)) # 리뷰 개수 출력

200000


In [ ]:
# NULL 값 존재 유무
print(train_data.isnull().values.any())

True


In [ ]:
train_data = train_data.dropna(how = 'any') # Null 값이 존재하는 행 제거
print(train_data.isnull().values.any()) # Null 값이 존재하는지 확인

False


In [ ]:
print(len(train_data)) # 리뷰 개수 출력

199992


In [ ]:
# 정규 표현식을 통한 한글 외 문자 제거
train_data['document'] = train_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")

<ipython-input-29-d10eedfa8951>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  train_data['document'] = train_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")


In [ ]:
train_data[:5] # 상위 5개 출력

,id,document,label
0,8112052,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1
1,8132799,디자인을 배우는 학생으로 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산업...,1
2,4655635,폴리스스토리 시리즈는 부터 뉴까지 버릴께 하나도 없음 최고,1
3,9251303,와 연기가 진짜 개쩔구나 지루할거라고 생각했는데 몰입해서 봤다 그래 이런게 진짜 영화지,1
4,10067386,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화,1


In [ ]:
# 불용어 정의
stopwords = ['도', '는', '다', '의', '가', '이', '은', '한', '에', '하', '고', '을', '를', '인', '듯', '과', '와', '네', '들', '듯', '지', '임', '게']

In [ ]:
# 형태소 분석기 mecab을 사용한 토큰화 작업 (다소 시간 소요)
mecab = Mecab()
tokenized_data = []
for sentence in train_data['document']:
    temp_X = mecab.morphs(sentence) # 토큰화
    temp_X = [word for word in temp_X if not word in stopwords] # 불용어 제거
    tokenized_data.append(temp_X)

In [ ]:
print(tokenized_data[:3])

[['어릴', '때', '보', '지금', '다시', '봐도', '재밌', '어요', 'ㅋㅋ'], ['디자인', '배우', '학생', '으로', '외국', '디자이너', '그', '일군', '전통', '통해', '발전', '해', '문화', '산업', '부러웠', '는데', '사실', '우리', '나라', '에서', '그', '어려운', '시절', '끝', '까지', '열정', '지킨', '노라노', '같', '전통', '있', '어', '저', '같', '사람', '꿈', '꾸', '이뤄나갈', '수', '있', '다는', '것', '감사', '합니다'], ['폴리스', '스토리', '시리즈', '부터', '뉴', '까지', '버릴', '께', '하나', '없', '음', '최고']]


## 한국어 Word2Vec 훈련시키기

nested list를 데이터로 넣어주셔야 합니다.

[['나는', '사과를', 먹는다'], ['이', '영화', '는', '재밌어']]

In [ ]:
from gensim.models import Word2Vec
model = Word2Vec(sentences = tokenized_data, vector_size = 100, window = 5, min_count = 5, workers = 4, sg = 0)

In [ ]:
# 완성된 임베딩 매트릭스의 크기 확인
model.wv.vectors.shape

(18134, 100)

In [ ]:
print(model.wv.most_similar("최민식"))

[('안성기', 0.8506284952163696), ('박중훈', 0.8444860577583313), ('한석규', 0.8345860838890076), ('드니로', 0.8305990099906921), ('알파치노', 0.830447793006897), ('송강호', 0.8219809532165527), ('박신양', 0.8199564218521118), ('송윤아', 0.8179770708084106), ('채민서', 0.8072400689125061), ('워싱턴', 0.8065908551216125)]


In [ ]:
model.wv['최민식']

array([-0.04925654,  0.16742939, -0.1258184 , -0.15379745, -0.05557119,
       -0.34051067,  0.17916293,  0.07992572, -0.16940686, -0.03847189,
        0.00820069, -0.374319  , -0.2823335 ,  0.14795698, -0.12966186,
        0.11648722, -0.19396351, -0.02685751,  0.12883535, -0.3712039 ,
        0.26960754, -0.02462203,  0.15347315,  0.26440468, -0.15457596,
        0.32119527,  0.09209904, -0.24401169, -0.09286335,  0.02223355,
        0.26793042,  0.04757704,  0.27240452,  0.13247903, -0.08296611,
        0.36766228,  0.07628651,  0.23881419, -0.34368053, -0.46687084,
        0.09350122, -0.47026554,  0.14860412,  0.12596354,  0.40145287,
       -0.24854359, -0.17700417, -0.5434466 ,  0.2666855 , -0.03224896,
        0.05717401,  0.21860336,  0.02898853,  0.04849403,  0.02440621,
       -0.23334035,  0.00456365, -0.01062528,  0.2790986 , -0.07397725,
        0.28853127,  0.3673138 , -0.3402923 , -0.01661944, -0.16966435,
        0.08439063,  0.07483184,  0.24207872, -0.41856965,  0.30

In [ ]:
print(model.wv.most_similar("히어로"))

[('호러', 0.8639163374900818), ('슬래셔', 0.8536562323570251), ('고어', 0.8325884342193604), ('느와르', 0.8124414086341858), ('블록버스터', 0.8080453276634216), ('무비', 0.8044121265411377), ('로코', 0.7925965189933777), ('정통', 0.7905184626579285), ('괴수', 0.7875390648841858), ('최고봉', 0.7812705039978027)]


In [ ]:
# 영어 모델이 저장된 경로로 이동
%cd /content

/content


In [ ]:
from gensim.models import KeyedVectors
model.wv.save_word2vec_format('kor_w2v') # 모델 저장

## 영어 Word2Vec 시각화

강의에서는 설명하지 않았지만 참고하시라고 넣은 내용입니다.

eng_w2v라는 Word2Vec 모델이 이미 존재한다는 가정 하에 아래 커맨드를 수행합니다.

In [ ]:
!python -m gensim.scripts.word2vec2tensor --input eng_w2v --output eng_w2v

2023-08-27 03:25:08,535 - word2vec2tensor - INFO - running /usr/local/lib/python3.10/dist-packages/gensim/scripts/word2vec2tensor.py --input eng_w2v --output eng_w2v
2023-08-27 03:25:08,535 - keyedvectors - INFO - loading projection weights from eng_w2v
2023-08-27 03:25:09,680 - utils - INFO - KeyedVectors lifecycle event {'msg': 'loaded (21613, 100) matrix of type float32 from eng_w2v', 'binary': False, 'encoding': 'utf8', 'datetime': '2023-08-27T03:25:09.678627', 'gensim': '4.3.1', 'python': '3.10.12 (main, Jun 11 2023, 05:26:28) [GCC 11.4.0]', 'platform': 'Linux-5.15.109+-x86_64-with-glibc2.35', 'event': 'load_word2vec_format'}
2023-08-27 03:25:10,965 - word2vec2tensor - INFO - 2D tensor file saved to eng_w2v_tensor.tsv
2023-08-27 03:25:10,965 - word2vec2tensor - INFO - Tensor metadata file saved to eng_w2v_metadata.tsv
2023-08-27 03:25:10,966 - word2vec2tensor - INFO - finished running word2vec2tensor.py


커맨드를 수행하면 경로에 기존에 있던 eng_w2v 외에도 두 개의 파일이 생깁니다.



링크 : https://projector.tensorflow.org/  

이제 해당 링크로 이동!

위에 있는 Choose file 버튼을 누르고 eng_w2v_tensor.tsv 파일을 업로드하고, 아래에 있는 Choose file 버튼을 누르고 eng_w2v_metadata.tsv 파일을 업로드합니다. 두 파일을 업로드하면 임베딩 프로젝터에 학습했던 워드 임베딩 모델이 시각화됩니다.



## 한국어 Word2Vec 시각화하기

In [ ]:
!python -m gensim.scripts.word2vec2tensor --input kor_w2v --output kor_w2v

2023-08-27 03:25:23,618 - word2vec2tensor - INFO - running /usr/local/lib/python3.10/dist-packages/gensim/scripts/word2vec2tensor.py --input kor_w2v --output kor_w2v
2023-08-27 03:25:23,618 - keyedvectors - INFO - loading projection weights from kor_w2v
2023-08-27 03:25:24,659 - utils - INFO - KeyedVectors lifecycle event {'msg': 'loaded (18134, 100) matrix of type float32 from kor_w2v', 'binary': False, 'encoding': 'utf8', 'datetime': '2023-08-27T03:25:24.657785', 'gensim': '4.3.1', 'python': '3.10.12 (main, Jun 11 2023, 05:26:28) [GCC 11.4.0]', 'platform': 'Linux-5.15.109+-x86_64-with-glibc2.35', 'event': 'load_word2vec_format'}
2023-08-27 03:25:25,747 - word2vec2tensor - INFO - 2D tensor file saved to kor_w2v_tensor.tsv
2023-08-27 03:25:25,747 - word2vec2tensor - INFO - Tensor metadata file saved to kor_w2v_metadata.tsv
2023-08-27 03:25:25,747 - word2vec2tensor - INFO - finished running word2vec2tensor.py


# FastText

## Word2Vec의 OOV 문제 확인해보기

OOV 문제(Out-Of-Vocabulary Problem) : Vocabulary에 존재하지 않는 단어가 등장하는 문제

In [ ]:
loaded_model = KeyedVectors.load_word2vec_format("eng_w2v") # Word2Vec 모델 로드

In [ ]:
model_result = loaded_model.most_similar("overacting")
print(model_result)

KeyError: ignored

In [ ]:
model_result = loaded_model.most_similar("memory")
print(model_result)

[('imagination', 0.6987040638923645), ('perception', 0.6984149813652039), ('reputation', 0.673920750617981), ('vision', 0.6715447902679443), ('consciousness', 0.6642743349075317), ('body', 0.6496174931526184), ('brain', 0.6496058106422424), ('reflection', 0.6454155445098877), ('logic', 0.6384631395339966), ('function', 0.6273216605186462)]


In [ ]:
model_result = loaded_model.most_similar("memorry")
print(model_result)

KeyError: ignored

In [ ]:
model_result = loaded_model.most_similar("electrofishing")
print(model_result)

KeyError: ignored

## FastText로 같은 단어에 대해서 테스트해보기

In [ ]:
from gensim.models import FastText
fasttext_model = FastText(result, vector_size=100, window=5, min_count=5, workers=4, sg=1)

In [ ]:
fasttext_model.wv.most_similar('overacting')

[('subtracting', 0.888767659664154),
 ('distracting', 0.8780357241630554),
 ('contracting', 0.8739362359046936),
 ('interacting', 0.86323082447052),
 ('overarching', 0.8603273630142212),
 ('manipulating', 0.8522518277168274),
 ('extracting', 0.8477192521095276),
 ('impacting', 0.8462916612625122),
 ('overeating', 0.8406209349632263),
 ('dissecting', 0.836780309677124)]

In [ ]:
fasttext_model.wv.most_similar('memorry')

[('memo', 0.8252053260803223),
 ('forgery', 0.7877283692359924),
 ('nemo', 0.7820053100585938),
 ('memoir', 0.7686251401901245),
 ('rehearsal', 0.7566522359848022),
 ('forgot', 0.7529147863388062),
 ('utero', 0.7459688186645508),
 ('memorize', 0.7429377436637878),
 ('rehearse', 0.7407183647155762),
 ('emory', 0.7380470037460327)]

In [ ]:
fasttext_model.wv.most_similar("electrofishing")

[('electrolyte', 0.861903965473175),
 ('electrolux', 0.8607944846153259),
 ('electro', 0.8512719869613647),
 ('electroshock', 0.8457664847373962),
 ('electroencephalogram', 0.8450732827186584),
 ('airbag', 0.8301453590393066),
 ('electrogram', 0.829261839389801),
 ('airbus', 0.82756507396698),
 ('electrochemical', 0.8266757130622864),
 ('electric', 0.8224183917045593)]

# 자모 단위 FastText

In [ ]:
# 한글 자모 단위 처리 패키지 설치
!pip install hgtk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# fasttext 설치
!git clone https://github.com/facebookresearch/fastText.git
%cd fastText
!make
!pip install .

Cloning into 'fastText'...
remote: Enumerating objects: 3930, done.
remote: Counting objects: 100% (964/964), done.
remote: Compressing objects: 100% (139/139), done.
remote: Total 3930 (delta 873), reused 825 (delta 825), pack-reused 2966
Receiving objects: 100% (3930/3930), 8.24 MiB | 14.65 MiB/s, done.
Resolving deltas: 100% (2505/2505), done.
/content/fastText
c++ -pthread -std=c++11 -march=native -O3 -funroll-loops -DNDEBUG -c src/args.cc
c++ -pthread -std=c++11 -march=native -O3 -funroll-loops -DNDEBUG -c src/autotune.cc
c++ -pthread -std=c++11 -march=native -O3 -funroll-loops -DNDEBUG -c src/matrix.cc
c++ -pthread -std=c++11 -march=native -O3 -funroll-loops -DNDEBUG -c src/dictionary.cc
c++ -pthread -std=c++11 -march=native -O3 -funroll-loops -DNDEBUG -c src/loss.cc
c++ -pthread -std=c++11 -march=native -O3 -funroll-loops -DNDEBUG -c src/productquantizer.cc
c++ -pthread -std=c++11 -march=native -O3 -funroll-loops -DNDEBUG -c src/densematrix.cc
c++ -pthread -std=c++11 -march=nati

## hgtk 튜토리얼

In [ ]:
import hgtk

In [ ]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/bab2min/corpus/master/sentiment/naver_shopping.txt", filename="ratings_total.txt")

('ratings_total.txt', <http.client.HTTPMessage at 0x7f2e448b2820>)

In [ ]:
total_data = pd.read_table('ratings_total.txt', names=['ratings', 'reviews'])
print('전체 리뷰 개수 :',len(total_data)) # 전체 리뷰 개수 출력

전체 리뷰 개수 : 200000


위의 링크로부터 전체 데이터에 해당하는 ratings_total.txt를 다운로드합니다. 해당 데이터에는 열제목이 별도로 없습니다. 그래서 임의로 두 개의 열제목인 'ratings'와 'reviews'를 추가해주겠습니다.

In [ ]:
total_data[:5]

,ratings,reviews
0,5,배공빠르고 굿
1,2,택배가 엉망이네용 저희집 밑에층에 말도없이 놔두고가고
2,5,아주좋아요 바지 정말 좋아서2개 더 구매했어요 이가격에 대박입니다. 바느질이 조금 ...
3,2,선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니다. 전...
4,5,민트색상 예뻐요. 옆 손잡이는 거는 용도로도 사용되네요 ㅎㅎ


한글의 자모를 처리하는 패키지인 hgtk를 사용하기에 앞서 간단히 사용법을 익혀봅시다. hgtk의 checker를 사용하면 입력이 한글인지 아닌지를 판단하여 True 또는 False를 리턴합니다.

In [ ]:
# 한글인지 체크
hgtk.checker.is_hangul('ㄱ')

True

In [ ]:
# 한글인지 체크
hgtk.checker.is_hangul('28')

False

hgtk의 letter를 사용하면 음절을 자모 단위로 분리하거나, 자모의 시퀀스를 다시 음절로 조합할 수 있습니다. 이는 각각 decompose와 compose로 가능합니다.

In [ ]:
# 음절을 초성, 중성, 종성으로 분해
hgtk.letter.decompose('남')

('ㄴ', 'ㅏ', 'ㅁ')

In [ ]:
# 초성, 중성을 결합
hgtk.letter.compose('ㄴ', 'ㅏ')

'나'

In [ ]:
# 초성, 중성, 종성을 결합
hgtk.letter.compose('ㄴ', 'ㅏ', 'ㅁ')

'남'

한글이 아닌 입력이 들어오거나 음절로 조합할 수 없는 경우 NotHangulException을 발생시킵니다.



In [ ]:
# 한글이 아닌 입력에 대해서는 에러 발생.
hgtk.letter.decompose('1')

NotHangulException: ignored

In [ ]:
# 결합할 수 없는 상황에서는 에러 발생
hgtk.letter.compose('ㄴ', 'ㅁ', 'ㅁ')

NotHangulException: ignored

## 데이터 전처리

위에서 사용했던 hgtk.letter.decompose()를 사용하여 특정 단어가 들어오면 이를 초성, 중성, 종성으로 나누는 함수 word_to_jamo를 구현합니다. 단, 종성이 없는 경우에는 해당 위치에 종성이 없었다는 것을 표시해주기 위해서 종성의 위치에 특수문자 '-'를 넣어주었습니다.

In [ ]:
def word_to_jamo(token):
  def to_special_token(jamo):
    if not jamo:
      return '-'
    else:
      return jamo

  decomposed_token = ''
  for char in token:
    try:
      # char(음절)을 초성, 중성, 종성으로 분리
      cho, jung, jong = hgtk.letter.decompose(char)

      # 자모가 빈 문자일 경우 특수문자 -로 대체
      cho = to_special_token(cho)
      jung = to_special_token(jung)
      jong = to_special_token(jong)
      decomposed_token = decomposed_token + cho + jung + jong

    # 만약 char(음절)이 한글이 아닐 경우 자모를 나누지 않고 추가
    except Exception as exception:
      if type(exception).__name__ == 'NotHangulException':
        decomposed_token += char

  # 단어 토큰의 자모 단위 분리 결과를 추가
  return decomposed_token

해당 함수에 임의의 단어 '남동생'을 넣어 정상적으로 분리하는지 테스트해봅시다.



In [ ]:
word_to_jamo('남동생')

'ㄴㅏㅁㄷㅗㅇㅅㅐㅇ'

'남동생'이 'ㄴㅏㅁㄷㅗㅇㅅㅐㅇ'으로 분리된 것을 확인할 수 있습니다. 이번에는 임의의 단어 '여동생'을 넣어서 테스트해봅시다.



In [ ]:
word_to_jamo('여동생')

'ㅇㅕ-ㄷㅗㅇㅅㅐㅇ'

'여동생'의 경우 여에 종성이 없으므로 종성의 위치에 특수문자 '-'가 대신 들어간 것을 확인할 수 있습니다. 단순 형태소 분석을 했을 경우와 형태소 분석 후에 다시 자모 단위로 분해하는 경우를 동일한 예문을 통해 비교해봅시다. 우선 단순 형태소 분석을 했을 경우입니다.

In [ ]:
mecab = Mecab()

In [ ]:
print(mecab.morphs('선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니다.'))

['선물', '용', '으로', '빨리', '받', '아서', '전달', '했어야', '하', '는', '상품', '이', '었', '는데', '머그', '컵', '만', '와서', '당황', '했', '습니다', '.']


우리가 일반적으로 봐왔던 형태소 분석 결과입니다. word_to_jamo 함수를 형태소 분석 후 호출하도록 하여 형태소 토큰들을 자모 단위로 분해하는 함수 tokenize_by_jamo를 정의합니다.

In [ ]:
def tokenize_by_jamo(s):
    return [word_to_jamo(token) for token in mecab.morphs(s)]

형태소 분석 후 자모 단위로 다시 한 번 분해한 경우입니다.



In [ ]:
print(tokenize_by_jamo('선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니다.'))

['ㅅㅓㄴㅁㅜㄹ', 'ㅇㅛㅇ', 'ㅇㅡ-ㄹㅗ-', 'ㅃㅏㄹㄹㅣ-', 'ㅂㅏㄷ', 'ㅇㅏ-ㅅㅓ-', 'ㅈㅓㄴㄷㅏㄹ', 'ㅎㅐㅆㅇㅓ-ㅇㅑ-', 'ㅎㅏ-', 'ㄴㅡㄴ', 'ㅅㅏㅇㅍㅜㅁ', 'ㅇㅣ-', 'ㅇㅓㅆ', 'ㄴㅡㄴㄷㅔ-', 'ㅁㅓ-ㄱㅡ-', 'ㅋㅓㅂ', 'ㅁㅏㄴ', 'ㅇㅘ-ㅅㅓ-', 'ㄷㅏㅇㅎㅘㅇ', 'ㅎㅐㅆ', 'ㅅㅡㅂㄴㅣ-ㄷㅏ-', '.']


자모 단위 FastText에서는 위와 같이 각 형태소 분석 결과 토큰들이 추가적으로 자모 단위로 분해된 토큰들을 가지고 학습을 하게 됩니다. 전체 데이터에 대해서 위의 자모 단위 토큰화를 적용해봅시다.

In [ ]:
from tqdm import tqdm

In [ ]:
tokenized_data = []

for sample in total_data['reviews'].to_list():
    tokenzied_sample = tokenize_by_jamo(sample) # 자소 단위 토큰화
    tokenized_data.append(tokenzied_sample)

첫번째 샘플을 출력해봅시다.



In [ ]:
tokenized_data[0]

['ㅂㅐ-ㄱㅗㅇ', 'ㅃㅏ-ㄹㅡ-', 'ㄱㅗ-', 'ㄱㅜㅅ']

'배공빠르고 굿'이라는 기존 샘플이 형태소 분석 후에는 ['배공', '빠르', '고', '굿']으로 분해되었으며, 이를 다시 자모 단위로 나누면서 ['ㅂㅐ-ㄱㅗㅇ', 'ㅃㅏ-ㄹㅡ-', 'ㄱㅗ-', 'ㄱㅜㅅ']라는 결과가 됩니다. 그런데 이렇게 바꾸고나니 원래 단어가 무엇이었는지 알아보기 힘들다는 문제가 있습니다. 출력했을 때, 사용자가 기존의 단어가 무엇이었는지를 쉽게 알아보기 위해 초성, 중성, 종성을 입력받으면 역으로 단어로 바꿔주는 jamo_to_word 함수를 구현합니다.

In [ ]:
def jamo_to_word(jamo_sequence):
  tokenized_jamo = []
  index = 0

  # 1. 초기 입력
  # jamo_sequence = 'ㄴㅏㅁㄷㅗㅇㅅㅐㅇ'

  while index < len(jamo_sequence):
    # 문자가 한글(정상적인 자모)이 아닐 경우
    if not hgtk.checker.is_hangul(jamo_sequence[index]):
      tokenized_jamo.append(jamo_sequence[index])
      index = index + 1

    # 문자가 정상적인 자모라면 초성, 중성, 종성을 하나의 토큰으로 간주.
    else:
      tokenized_jamo.append(jamo_sequence[index:index + 3])
      index = index + 3

  # 2. 자모 단위 토큰화 완료
  # tokenized_jamo : ['ㄴㅏㅁ', 'ㄷㅗㅇ', 'ㅅㅐㅇ']

  word = ''
  try:
    for jamo in tokenized_jamo:

      # 초성, 중성, 종성의 묶음으로 추정되는 경우
      if len(jamo) == 3:
        if jamo[2] == "-":
          # 종성이 존재하지 않는 경우
          word = word + hgtk.letter.compose(jamo[0], jamo[1])
        else:
          # 종성이 존재하는 경우
          word = word + hgtk.letter.compose(jamo[0], jamo[1], jamo[2])
      # 한글이 아닌 경우
      else:
        word = word + jamo

  # 복원 중(hgtk.letter.compose) 에러 발생 시 초기 입력 리턴.
  # 복원이 불가능한 경우 예시) 'ㄴ!ㅁㄷㅗㅇㅅㅐㅇ'
  except Exception as exception:
    if type(exception).__name__ == 'NotHangulException':
      return jamo_sequence

  # 3. 단어로 복원 완료
  # word : '남동생'

  return word

해당 함수의 내부 동작 방식을 설명하기 위해 'ㄴㅏㅁㄷㅗㅇㅅㅐㅇ'이라는 임의의 입력이 들어왔을 때를 가정해봅시다. 초기 입력이 들어왔을 때는 jamo_sequence라는 변수에 저장되어져 있습니다. while 문 내부에서는 jamo_sequences의 각 문자에 대해서 세 개씩 분리하여 초성, 중성, 종성을 하나의 묶음으로 간주합니다. while문을 지나고나면 'ㄴㅏㅁㄷㅗㅇㅅㅐㅇ'이라는 문자열은 ['ㄴㅏㅁ', 'ㄷㅗㅇ', 'ㅅㅐㅇ']이라는 리스트로 변환이 되며, 해당 리스트는 tokenized_jamo라는 변수에 저장되어져 있습니다. 그리고 각 리스트의 원소를 hgtk.letter.compose()의 입력으로 넣어 기존의 음절로 복원합니다. 결과적으로 '남동생'이라는 단어로 복원되고 해당 함수는 '남동생'을 최종 결과로서 리턴합니다.

실제로 'ㄴㅏㅁㄷㅗㅇㅅㅐㅇ'을 입력으로 넣어 결과를 확인해봅시다.



In [ ]:
jamo_to_word('ㄴㅏㅁㄷㅗㅇㅅㅐㅇ')

'남동생'

## FastText

자모 단위로 토큰화 된 데이터를 가지고 FastText를 학습시켜봅시다.



In [ ]:
import fasttext

FastText 학습을 위해서 기존 훈련 데이터를 txt 파일 형식으로 저장해야합니다.



In [ ]:
with open('tokenized_data.txt', 'w') as out:
  for line in tqdm(tokenized_data, unit=' line'):
    out.write(' '.join(line) + '\n')

100%|██████████| 200000/200000 [00:00<00:00, 473187.85 line/s]


두 가지 모델 Skip-gram과 CBoW 중 CBoW를 선택했습니다.



In [ ]:
model = fasttext.train_unsupervised('tokenized_data.txt', model='cbow')

In [ ]:
model.save_model("fasttext.bin")

In [ ]:
model = fasttext.load_model("fasttext.bin")

학습이 완료되었습니다. 임의로 '남동생'이라는 단어의 벡터값을 확인해봅시다. 주의할 점은 학습 시 자모 단위로 분해하였기 때문에 모델에서 벡터값을 확인할 때도 자모 단위로 분해 후에 입력으로 사용해야 합니다.

In [ ]:
model[word_to_jamo('남동생')] # 'ㄴㅏㅁㄷㅗㅇㅅㅐㅇ'

array([-0.75121707, -0.09777343, -0.06479282,  0.3878322 , -0.3629536 ,
        0.444919  ,  0.17368399, -0.6492414 , -0.1460425 , -0.6556199 ,
        0.47561774,  0.15500426, -0.07597088,  0.6838326 , -0.12038054,
       -0.05893848,  0.80528766,  0.41876066,  0.5232016 , -0.76272345,
       -0.38435844, -0.48213163,  0.47253323,  0.42145985, -0.40317082,
        0.1875647 ,  0.40562132,  0.0864673 ,  0.4318442 ,  0.41785347,
        0.99820423, -1.1779842 ,  0.02945196,  0.33049268,  1.0954536 ,
        0.03768162,  0.44268563, -1.095168  , -0.23283842,  0.5355596 ,
        0.6794727 ,  0.47900698, -0.60658246,  1.2562438 , -0.24471879,
        0.46079734, -0.01676963, -0.17489414, -0.43604985,  0.34543806,
       -0.15913598,  0.2918002 ,  0.8731358 ,  1.0437014 , -0.6025581 ,
        0.2981697 ,  0.53655666, -0.05084229,  0.05335928,  0.2421371 ,
        0.17908356, -0.6660127 ,  0.934657  ,  0.09646372, -0.6387341 ,
       -0.25673625, -0.77488786, -0.07015391, -0.37226695, -0.55

남동생 '벡터'와 가장 유사도가 높은 벡터들을 뽑아봅시다. 이는 get_nearest_neighbors()를 사용하여 가능합니다. 두번째 인자인 k값으로 10을 주면, 가장 유사한 벡터 상위 10개를 출력합니다.

In [ ]:
model.get_nearest_neighbors(word_to_jamo('남동생'), k=10)

[(0.8809508681297302, 'ㄷㅗㅇㅅㅐㅇ'),
 (0.8286653161048889, 'ㄴㅏㅁㅊㅣㄴ'),
 (0.7495564818382263, 'ㄴㅏㅁㅍㅕㄴ'),
 (0.7464686036109924, 'ㅅㅐㅇㅇㅣㄹ'),
 (0.735321581363678, 'ㅊㅣㄴㄱㅜ-'),
 (0.7235512733459473, 'ㄴㅏㅁㅇㅏ-'),
 (0.7162966132164001, 'ㅈㅗ-ㅋㅏ-'),
 (0.7039785385131836, 'ㄴㅏㄴㅅㅐㅇ'),
 (0.6938914656639099, 'ㄴㅏㅁㅁㅐ-'),
 (0.6907734870910645, 'ㅇㅓㄴㄴㅣ-')]

그런데 출력으로 나오는 벡터들도 자모 단위로 분해해서 나오기 때문에 읽기가 어렵습니다. 이전에 만들어준 jamo_to_word 함수를 사용하여 출력 결과를 좀 더 깔끔하게 확인할 수 있습니다.

In [ ]:
def transform(word_sequence):
  return [(jamo_to_word(word), similarity) for (similarity, word) in word_sequence]

In [ ]:
print(transform(model.get_nearest_neighbors(word_to_jamo('남동생'), k=10)))

[('동생', 0.8809508681297302), ('남친', 0.8286653161048889), ('남편', 0.7495564818382263), ('생일', 0.7464686036109924), ('친구', 0.735321581363678), ('남아', 0.7235512733459473), ('조카', 0.7162966132164001), ('난생', 0.7039785385131836), ('남매', 0.6938914656639099), ('언니', 0.6907734870910645)]


In [ ]:
print(transform(model.get_nearest_neighbors(word_to_jamo('남동쉥'), k=10)))

[('남동생', 0.903592586517334), ('남친', 0.8046611547470093), ('남매', 0.7930417656898499), ('남짓', 0.762987494468689), ('남김', 0.7530947327613831), ('남긴', 0.7364609241485596), ('남여', 0.7232005000114441), ('남아', 0.7184900045394897), ('남겼', 0.7151230573654175), ('남녀', 0.7114916443824768)]


In [ ]:
print(transform(model.get_nearest_neighbors(word_to_jamo('남동셍ㅋ'), k=10)))

[('남동생', 0.8321648240089417), ('남친', 0.7351190447807312), ('남김', 0.7152878642082214), ('남짓', 0.6808348298072815), ('남매', 0.6775488257408142), ('남길', 0.6609652638435364), ('남긴', 0.6552697420120239), ('남녀', 0.647351861000061), ('남여', 0.6399891972541809), ('남겼', 0.6360887885093689)]


In [ ]:
print(transform(model.get_nearest_neighbors(word_to_jamo('난동생'), k=10)))

[('남동생', 0.8605595231056213), ('난생', 0.8415924906730652), ('동생', 0.790320098400116), ('남편', 0.7590644359588623), ('남아', 0.7492496967315674), ('남친', 0.7343378067016602), ('중학생', 0.7026507258415222), ('남매', 0.6978760957717896), ('학생', 0.6911318898200989), ('반신욕', 0.6841548681259155)]


In [ ]:
print(transform(model.get_nearest_neighbors(word_to_jamo('낫동생'), k=10)))

[('남동생', 0.9290178418159485), ('동생', 0.88860684633255), ('남편', 0.7612314820289612), ('난생', 0.7438672184944153), ('남친', 0.7399165630340576), ('조카', 0.7122743725776672), ('중학생', 0.7097246646881104), ('친구', 0.7055485248565674), ('남아', 0.7020242810249329), ('학생', 0.6935662031173706)]


In [ ]:
print(transform(model.get_nearest_neighbors(word_to_jamo('납동생'), k=10)))

[('남동생', 0.9063751101493835), ('동생', 0.8477023243904114), ('난생', 0.77754145860672), ('남편', 0.7669044733047485), ('남친', 0.7531878352165222), ('남아', 0.7262435555458069), ('중학생', 0.7041069269180298), ('조카', 0.697344958782196), ('나눴', 0.6933421492576599), ('고등학생', 0.6913762092590332)]


In [ ]:
print(transform(model.get_nearest_neighbors(word_to_jamo('냚동생'), k=10)))

[('동생', 0.9683497548103333), ('남동생', 0.8915597796440125), ('조카', 0.781344473361969), ('친구', 0.7688440680503845), ('생일', 0.7618958353996277), ('언니', 0.7535653114318848), ('학생', 0.7456444501876831), ('중학생', 0.7425476312637329), ('어린이집', 0.7221997380256653), ('초등학생', 0.7162642478942871)]


In [ ]:
print(transform(model.get_nearest_neighbors(word_to_jamo('제품^^'), k=10)))

[('제품', 0.9403702616691589), ('제풍', 0.8477377891540527), ('반제품', 0.7876006960868835), ('완제품', 0.7851003408432007), ('상품', 0.7712583541870117), ('최상품', 0.7655972838401794), ('타제품', 0.7592073082923889), ('재품', 0.7041335105895996), ('화학제품', 0.6883261799812317), ('상품명', 0.6684249043464661)]
